In [ ]:
# Needed for every metric
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.0 MB/s 
     |████████████████████████████████| 115 kB 66.4 MB/s 
     |████████████████████████████████| 141 kB 70.3 MB/s 
     |████████████████████████████████| 212 kB 62.9 MB/s 
     |████████████████████████████████| 101 kB 14.8 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 127 kB 65.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# necessary
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.5 MB/s 
     |████████████████████████████████| 4.7 MB 10.1 MB/s 
     |████████████████████████████████| 6.6 MB 51.5 MB/s 


In [ ]:
!git clone https://github.com/drehero/geneval

Cloning into 'geneval'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 225 (delta 9), reused 17 (delta 7), pack-reused 205
Receiving objects: 100% (225/225), 203.27 KiB | 2.23 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [ ]:
import datasets
import numpy as np

from geneval.geneval.data.wmt import WMT18

In [ ]:
# kendall score
# https://github.com/Tiiiger/bert_score/blob/master/reproduce/get_wmt18_seg_results.py#L53

def kendall_score(scores_better, scores_worse):
    total = len(scores_better)
    correct = np.sum(np.array(scores_better) > np.array(scores_worse))
    incorrect = total - correct
    return (correct - incorrect)/total

### BERTscore

In [ ]:
bert_score = datasets.load_metric("bertscore")

In [ ]:
results = {}
for lang_pair in ["cs-en", "de-en", "et-en", "fi-en", "ru-en", "tr-en", "zh-en"]:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")
    # compute scores
    scores_better = bert_score.compute(
        predictions=wmt.translations_better,
        references=wmt.references,
        lang="en"
    )
    scores_worse = bert_score.compute(
        predictions=wmt.translations_worse,
        references=wmt.references,
        lang="en"
    )
    ks = kendall_score(scores_better["f1"], scores_worse["f1"])
    results[lang_pair] = ks

Expected

{'cs-en': 0.4140900195694716,
 'de-en': 0.5548187274292837,
 'et-en': 0.4033603074698965,
 'fi-en': 0.30610940695296524,
 'ru-en': 0.354479046520569,
 'tr-en': 0.3020527859237537,
 'zh-en': 0.2647420331564589}

In [ ]:
results

{'cs-en': 0.4140900195694716,
 'de-en': 0.5548187274292837,
 'et-en': 0.4033603074698965,
 'fi-en': 0.30610940695296524,
 'ru-en': 0.354479046520569,
 'tr-en': 0.3020527859237537,
 'zh-en': 0.2647420331564589}

### COMET

In [ ]:
pip install unbabel-comet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 49.6 MB/s 
     |████████████████████████████████| 585 kB 63.5 MB/s 
     |████████████████████████████████| 116 kB 72.2 MB/s 
     |████████████████████████████████| 409 kB 70.9 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 9.5 MB 50.5 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
comet = datasets.load_metric("comet")

wmt20-comet-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


In [ ]:
source = ["Dem Feuer konnte Einhalt geboten werden", "Schulen und Kindergärten wurden eröffnet."]
hypothesis = ["The fire could be stopped", "Schools and kindergartens were open"]
reference = ["They were able to control the fire.", "Schools and kindergartens opened"]
comet_score = comet.compute(predictions=hypothesis, references=reference, sources=source)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
comet_score

{'mean_score': 0.5529160350561142,
 'scores': [0.19016960263252258, 0.9156624674797058]}

In [ ]:
comet_results = {}
for lang_pair in ["cs-en", "de-en", "et-en", "fi-en", "ru-en", "tr-en", "zh-en"]:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")
    # compute scores
    scores_better = comet.compute(
        sources = wmt.sources,
        predictions = wmt.translations_better,
        references = wmt.references
    )
    scores_worse = comet.compute(
        sources = wmt.sources,
        predictions = wmt.translations_worse,
        references = wmt.references
    )
    ks = kendall_score(scores_better["scores"], scores_worse["scores"])
    comet_results[lang_pair] = ks

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_ligh

In [ ]:
comet_results

{'cs-en': 0.500587084148728,
 'de-en': 0.6131652337073165,
 'et-en': 0.47624336665432554,
 'fi-en': 0.39851738241308793,
 'ru-en': 0.42541330257593235,
 'tr-en': 0.4055131964809384,
 'zh-en': 0.35281949815630903}

Comet results: {'cs-en': 0.500587084148728,
 'de-en': 0.6131652337073165,
 'et-en': 0.47624336665432554,
 'fi-en': 0.39851738241308793,
 'ru-en': 0.42541330257593235,
 'tr-en': 0.4055131964809384,
 'zh-en': 0.35281949815630903}